In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np
import numpy.random as nprand
import pandas as pd
import random
import scipy.stats as spstats
%matplotlib inline

from loomio import *
from netdelib import *
from socialchoice import *
from timeseries import *

## Evolution - Group Level

In [ ]:
class NetDelibEvoInitialMedian(object):
    def __init__(self, df_score):
        self.df_control = df_score[df_score.treatment == 1]
        self.df_random = df_score[df_score.treatment == 2]

    def plot_kendall(self):
        control = self.kendall(self.df_control)
        random = self.kendall(self.df_random)

        plt.plot(control[0], control[1], '.', color="#00007f7f", markersize=30, label='Control')
        plt.plot(random[0], random[1], '.', color="#007f007f", markersize=30, label='Random-Pod')
        plt.title('Distance to Initial Median (Kendall)')
        plt.xlabel('Initial Distance')
        plt.ylabel('Final Distance')
        plt.xlim([0,14])
        plt.ylim([0,14])
    
    def plot_ballot(self):
        control = self.ballot(self.df_control)
        random = self.ballot(self.df_random)

        plt.plot(control[0], control[1], '.', color="#00007f7f", markersize=30, label='Control')
        plt.plot(random[0], random[1], '.', color="#007f007f", markersize=30, label='Random-Pod')
        plt.title('Distance to Initial Median (Ballot)')
        plt.xlabel('Initial Distance')
        plt.ylabel('Final Distance')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
    
    def kendall(self, df_treatment):

        # Calculate sequence collection and profiles
        collection = make_preference_sequence_collection(df_treatment)
        profiles = ProfileSequence.from_preference_sequence_collection(collection)
                          
        # Find initial Kemeny-Young profiles
        ky_init = make_kemeny_young_set(df_treatment, 0)

        # Find mean distance to initial K-Y profile
        init_kendall = []
        final_kendall = []
        participant_ids = collection.participant_ids()
        for participant_id in participant_ids:
            sequence = collection[participant_id]
            participant_init = []
            participant_final = []
            for ky in ky_init:
                kyp = Preference(ky)
                participant_init.append(kyp.kendall_tau(sequence[0]))
                participant_final.append(kyp.kendall_tau(sequence[3]))
            init_kendall.append(np.mean(participant_init))
            final_kendall.append(np.mean(participant_final))
        return (init_kendall, final_kendall)

    def ballot(self, df_treatment):

        # Calculate sequence collection and profiles
        collection = make_preference_sequence_collection(df_treatment)
        profiles = ProfileSequence.from_preference_sequence_collection(collection)
                          
        # Find initial weighted swap median profiles
        median_init = make_ballot_set(df_treatment, 0)

        # Find mean distance to initial median profile
        init_ballot = []
        final_ballot = []
        participant_ids = collection.participant_ids()
        for participant_id in participant_ids:
            participant_init = []
            participant_final = []
            sequence = collection[participant_id]
            for ranking in median_init:
                pref = Preference(ranking)
                participant_init.append(pref.ballot_distance(sequence[0]))
                participant_final.append(pref.ballot_distance(sequence[3]))
            init_ballot.append(np.mean(participant_init))
            final_ballot.append(np.mean(participant_final))
        return (init_ballot, final_ballot)




In [ ]:
df_score, proposal_map, proposal_rev_map = load_loomio_score('results/results_2_3.tsv')
df_score = fill_attrition(df_score)

netdelib = NetDelibEvoInitialMedian(df_score)

In [ ]:
# Blue is control 
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
netdelib.plot_kendall()
plt.subplot(1,2,2)
netdelib.plot_ballot()
plt.tight_layout()

In [ ]:
control_kendall = netdelib.kendall(netdelib.df_control)
random_kendall = netdelib.kendall(netdelib.df_random)

control_diff = []
random_diff = []

for i in range(len(control_kendall[0])):
    control_diff.append(control_kendall[1][i] - control_kendall[0][i])
for i in range(len(random_kendall[0])):
    random_diff.append(random_kendall[1][i] - random_kendall[0][i])

plt.hist([control_diff, random_diff])
t, p = spstats.ttest_ind(control_diff, random_diff)
print(t, p)

In [ ]:
control_ballot = netdelib.ballot(netdelib.df_control)
random_ballot = netdelib.ballot(netdelib.df_random)

control_diff = []
random_diff = []

for i in range(len(control_ballot[0])):
    control_diff.append(control_ballot[1][i] - control_ballot[0][i])
for i in range(len(random_ballot[0])):
    random_diff.append(random_ballot[1][i] - random_ballot[0][i])

plt.hist([control_diff, random_diff], label=['Control', 'Random-Pod'])
t, p = spstats.ttest_ind(control_diff, random_diff)
print(t, p)
plt.legend()